## Scenario Visualization
The purpose of this notebook is to visualize the projects in a scenario that you constructed.

In [64]:
from pathlib import Path
import pandas as pd
import network_wrangler as wr

from ipywidgets import Output, HTML
from IPython.display import display

%config IPCompleter.greedy=True
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 1000)

import logging

logger = logging.getLogger("WranglerLogger")
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.INFO)

### Define Base Files, Networks, and Scenario

In [65]:
STPAUL_DIR = Path.cwd().parent / "examples" / "stpaul"


In [66]:
road_net = wr.load_roadway_from_dir(STPAUL_DIR)
transit_net = wr.load_transit(STPAUL_DIR)

Skipping field outboundReferenceIds: unsupported OGR type: 5
Skipping field inboundReferenceIds: unsupported OGR type: 5


In [67]:
base_scenario = {
    "road_net": road_net,
    "transit_net": transit_net,
}

### Build a Scenario by Specifying Specific Project Cards

In [68]:
BUILD_CARD_PATHS = [
    STPAUL_DIR/"project_cards"/"road.prop_change.multiple.yml",
    STPAUL_DIR/"project_cards"/"road.prop_change.widen.yml",
    STPAUL_DIR/"project_cards"/"road.managed_lane.simple.yml",
    STPAUL_DIR/"project_cards"/"transit.prop_change.route_name_contains.yml",
]

In [69]:
my_scenario = wr.create_scenario(
    base_scenario=base_scenario, project_card_filepath=BUILD_CARD_PATHS
)
my_scenario.apply_all_projects()

Base_scenario doesn't contain ['road_net', 'transit_net', 'applied_projects', 'conflicts']
Creating a deep copy of db object.            This will NOT update any references (e.g. from TransitNetwork)


In [70]:
my_scenario.applied_projects

['improve express bus frequency',
 'test managed lane project',
 'widen minnehaha pkwy',
 '6th street transitway']

## Project Summaries

- summarize the current state of the scenario using `summary` property 
- review projects that are applied in the roadway network or transit network using the `projects` field of any applicable dataframe.

In [71]:
my_scenario.summary

{'name': '20241016095637',
 'applied_projects': ['improve express bus frequency',
  'test managed lane project',
  'widen minnehaha pkwy',
  '6th street transitway'],
 'prerequisites': {},
 'corequisites': {},
 'conflicts': {},
 'config': {'IDS': {'TRANSIT_SHAPE_ID_METHOD': 'scalar',
   'TRANSIT_SHAPE_ID_SCALAR': 1000000,
   'ROAD_SHAPE_ID_METHOD': 'scalar',
   'ROAD_SHAPE_ID_SCALAR': 1000,
   'ML_LINK_ID_METHOD': 'scalar',
   'ML_LINK_ID_RANGE': (950000, 999999),
   'ML_LINK_ID_SCALAR': 3000000,
   'ML_NODE_ID_METHOD': 'range',
   'ML_NODE_ID_RANGE': (950000, 999999),
   'ML_NODE_ID_SCALAR': 15000},
  'MODEL_ROADWAY': {'ML_OFFSET_METERS': -10,
   'ADDITIONAL_COPY_FROM_GP_TO_ML': [],
   'ADDITIONAL_COPY_TO_ACCESS_EGRESS': []},
  'CPU': {'EST_PD_READ_SPEED': {'csv': 0.03,
    'parquet': 0.005,
    'geojson': 0.03,
    'json': 0.15,
    'txt': 0.04}},
  'EDITS': {'EXISTING_VALUE_CONFLICT': 'warn',
   'OVERWRITE_SCOPED': 'conflicting'}}}

In [72]:
display_cols = ["name","lanes","projects"]
_links_df = my_scenario.road_net.links_df
links_with_projects = _links_df.loc[_links_df.projects!=""]
links_with_projects[display_cols]

,name,lanes,projects
model_link_id_idx,,,
7194,Minnehaha Avenue,3,"Widen Minnehaha Pkwy,"
7196,Minnehaha Avenue,3,"Widen Minnehaha Pkwy,"
7298,Minnehaha Avenue,3,"Widen Minnehaha Pkwy,"
7300,Minnehaha Avenue,3,"Widen Minnehaha Pkwy,"
8481,Minnehaha Avenue East,3,"Widen Minnehaha Pkwy,"
...,...,...,...
171978,Minnehaha Avenue,3,"Widen Minnehaha Pkwy,"
171979,Minnehaha Avenue,3,"Widen Minnehaha Pkwy,"
275245,Minnehaha Avenue,3,"Widen Minnehaha Pkwy,"


In [88]:
display_cols = ["name","lanes","projects"]
mymap = links_with_projects.explore(tiles="CartoDB positron", column="projects", cmap = "tab20", tooltip=display_cols)
mymap

In [92]:
_freq_df = my_scenario.transit_net.feed.frequencies
freq_with_projects = _freq_df.loc[_freq_df.projects!=""]
freq_with_projects


,trip_id,headway_secs,start_time,end_time,projects
0,14940701-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"
6,14943414-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"
7,14943415-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"
28,14946111-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"
30,14946257-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"
31,14946470-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"
32,14946471-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 09:00:00,2024-10-16 15:00:00,"Improve Express Bus Frequency,"
33,14946480-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"
34,14946521-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 09:00:00,2024-10-16 15:00:00,"Improve Express Bus Frequency,"
35,14947182-JUN19-MVS-BUS-Weekday-01,1800,2024-10-16 06:00:00,2024-10-16 09:00:00,"Improve Express Bus Frequency,"


In [75]:
# To see the trips that have projects applied to them, we can filter trips table by the affected trip_ids
display_cols = ["route_id","trip_id","trip_headsign","projects_freq"]
affected_trips = freq_with_projects.trip_id.unique()
_trips_df = my_scenario.transit_net.feed.trips
_affected_trips_df = _trips_df.loc[_trips_df.trip_id.isin(affected_trips)]
trips_with_projects = _affected_trips_df.merge(freq_with_projects[["trip_id","projects"]], on="trip_id", how="left", suffixes=("","_freq"))
trips_with_projects[display_cols]

,route_id,trip_id,trip_headsign,projects_freq
0,452-111,14940701-JUN19-MVS-BUS-Weekday-01,Westbound 452 Express / Minneapolis,"Improve Express Bus Frequency,"
1,294-111,14943414-JUN19-MVS-BUS-Weekday-01,Westbound 294 Express / St Paul,"Improve Express Bus Frequency,"
2,294-111,14943415-JUN19-MVS-BUS-Weekday-01,Eastbound 294 Express/St Crx P&R/Via 4 Front Tech,"Improve Express Bus Frequency,"
3,860-111,14946111-JUN19-MVS-BUS-Weekday-01,Southbound 860 Express/St Paul/Northtown,"Improve Express Bus Frequency,"
4,355-111,14946257-JUN19-MVS-BUS-Weekday-01,Westbound 355 Express / Minneapolis,"Improve Express Bus Frequency,"
5,94-111,14946470-JUN19-MVS-BUS-Weekday-01,Westbound 94 Express / Downtown / Minneapolis,"Improve Express Bus Frequency,"
6,94-111,14946471-JUN19-MVS-BUS-Weekday-01,Westbound 94 Express / Downtown / Minneapolis,"Improve Express Bus Frequency,"
7,94-111,14946480-JUN19-MVS-BUS-Weekday-01,Eastbound 94D Express / Dwtn St Paul / Union Depot,"Improve Express Bus Frequency,"
8,94-111,14946521-JUN19-MVS-BUS-Weekday-01,Eastbound 94D Express / Dwtn St Paul / Union Depot,"Improve Express Bus Frequency,"
9,365-111,14947182-JUN19-MVS-BUS-Weekday-01,Northbound 365 Express / Minneapolis,"Improve Express Bus Frequency,"


In [93]:
# And to see them on the map, we can filter the stoptimes table by the affected stop_ids and offset the geometries.

_stoptimes_gdf = my_scenario.transit_net.stop_time_links_gdf.explode("trip_id").reset_index()

affected_stoptimes = _stoptimes_gdf.loc[_stoptimes_gdf.trip_id.isin(affected_trips)]
affected_stoptimes['i'] = affected_stoptimes.groupby('index').cumcount() + 1
affected_stoptimes = affected_stoptimes.reset_index(drop=True)

# join the project info with stoptimes GDF
stoptimes_w_projects = affected_stoptimes.merge(freq_with_projects[["trip_id","projects", "headway_secs"]], on="trip_id", how="left", suffixes=("","_freq"))

# offset the geometries by trip_id so that they can be seen on the map
stoptimes_w_projects["offset"] = 0.00001 * (stoptimes_w_projects["i"]-1)
stoptimes_w_projects["offset_geometry"] = stoptimes_w_projects.apply(lambda x: x.geometry.offset_curve(x.offset), axis=1)
stoptimes_w_projects = stoptimes_w_projects.set_geometry("offset_geometry")

stoptimes_w_projects.explore(column="trip_id", m=mymap, cmap="Paired", tooltip=["trip_id","projects", "headway_secs"], layer_name="Transit Projects")